**Table of contents**<a id='toc0_'></a>    
- [Résumé](#toc1_)    
  - [Problématique](#toc1_1_)    
  - [Extrait](#toc1_2_)    
- [Introduction](#toc2_)    
  - [Prérequis et imports](#toc2_1_)    
  - [Fonctions spécifiques](#toc2_2_)    
  - [Fonctions de traitement](#toc2_3_)    
  - [Chargement des données](#toc2_4_)    
  - [Nettoyage des données](#toc2_5_)    
- [Pré-traitement](#toc3_)    
  - [Séparation du jeu de données](#toc3_1_)    
  - [Encodage, normalisation et imputation](#toc3_2_)    
- [Pipeline complet](#toc4_)    
  - [Paramétrage et préparation des données](#toc4_1_)    
  - [Définitions](#toc4_2_)    
  - [Application du preprocessing](#toc4_3_)    
- [Estimateurs](#toc5_)    
  - [Mesures comparatives](#toc5_1_)    
  - [Étalon : dummy regressor](#toc5_2_)    
  - [Linear Regression](#toc5_3_)    
  - [Support Vector Machines (SVM)](#toc5_4_)    
  - [Random Forest Regressor](#toc5_5_)    
  - [Gradient Boosting Regressor](#toc5_6_)    
  - [Scores d'entraînement](#toc5_7_)    
  - [Comparatif structurel des échantillons cibles](#toc5_8_)    
- [Explication des scores : recherche](#toc6_)    
  - [Constats initiaux](#toc6_1_)    
    - [Influence de l'échantillonnage](#toc6_1_1_)    
    - [Méthode pas à pas sur 10 random states](#toc6_1_2_)    
    - [Fuite possible dans le pré-traitement ou dans le pipeline ?](#toc6_1_3_)    
  - [Recherche pas à pas : scores (r²) selon les traitements](#toc6_2_)    
  - [Interprétation des résultats et retour sur le brief](#toc6_3_)    
  - [Conclusions de la recherche de fuite](#toc6_4_)    
  - [Échantillonnage : sensibilité aux outliers](#toc6_5_)    
  - [Corrélation : feature importance et poids des modèles](#toc6_6_)    
  - [Conclusion de l'explication des scores](#toc6_7_)    
- [Choix de l'estimateur et hyperparamétrage](#toc7_)    
  - [Validation croisée](#toc7_1_)    
- [Influence de l'ENERGYSTAR Score](#toc8_)    
- [Comparatif avec ratio rés / non-rés](#toc9_)    
- [Conclusion](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Résumé](#toc0_)

## <a id='toc1_1_'></a>[Problématique](#toc0_)

À partir des données provenant de la ville de Seattle, nous recherchons à prédire la consommation énergétique et l'émission de CO2 des bâtiments non destinés à l'habitation, tout en évaluant l'intérêt de l' "ENERGY STAR Score" dans les prédictions d'émissions.  
Tout nouveau bâtiment aura un premier relevé de référence la première année.  

✅ MISSION 1 → réaliser une **[analyse exploratoire des données](./P3_EDA.ipynb)**  

**À partir des relevés existants et des données structurelles** des bâtiments (taille, usage, date de construction, situation géographique, ...) tenter de :  

👉 MISSION 2 → **prédire les émissions de CO2** pour les bâtiments non mesurés et **évaluer l'intérêt de l'ENERGY STAR Score** pour la prédiction d'émissions (fastidieux à calculer, à intégrer dans la modélisation)  
MISSION 3 → **[prédire la consommation totale d'énergie](./P3_ML_2_cons.ipynb)** pour les bâtiments non mesurés  

## <a id='toc1_2_'></a>[Extrait](#toc0_)

Les choix techniques se sont portés sur la création d'un **pipeline de pré-traitement**, comprenant :
- les actions de nettoyage et de feature engineering prévues dans l'EDA précédente
- séparation train/val/test du jeu de données
- encodage one hot des variables catégorielles (`Neighborhood`)
- normalisation min-max des variables numériques (entre 0 et 1)
- imputation kNN(n=5) des valeurs manquantes (`ENERGYSTARScore`)

Ce pipeline a été utilisé pour **comparer divers estimateurs de régression sur les échantillons de test** grâce à la bibliothèque Scikit Learn, avec :
- une utilisation des métriques r², MAE, RMSE et du temps d'entraînement pour comparer les modèles
- un étalon (`sklearn.dummy.DummyRegressor`)
- une régression linéaire simple (`sklearn.linear_model.LinearRegression`)
- une régression par vecteur de support (`sklearn.svm.SVR`) avec noyau linéaire
- une forêt aléatoire (`sklearn.ensemble.RandomForestRegressor`)
- un renforcement de gradient (`sklearn.ensemble.GradientBoostingRegressor`)

Les **scores sans hyperparamétrage étant bien trop élevés**, une **recherche explicative** a été menée comme suit :
- influence importante de l'échantillonnage selon le `random state` ➡️ boucle sur 10 random states et moyenne
- élimination d'une source de fuite en aval de la séparation du jeu de données ➡️ intensification de la recherche dans les actions de nettoyage
- recherche de fuite possible avec une méthode reprenant pas à pas chaque étape du pipe pour sauvegarder les métriques
  - détails dans un [NoteBook dédié](./P3_data_leak_tests.ipynb)
  - influence très forte des variables énergétiques structurelles
  - montée du r² à la création des variables et à la filtration des individus
- relecture du brief de Douglas
- 1️⃣ conclusion : pas de fuite de données
- 2️⃣ sensibilité aux outliers confirmée, menant à un sur-apprentissage
  - solution possible : hyperparamétrage
  - pratique nécessaire : boucle d'échantillonnage pour l'apprentissage (type *cross validation*)
- 3️⃣ confirmation de la forte corrélation avec `NaturalGas_I(kBtu/sf)` par la feature importance et l'analyse des poids des modèles
  - pas de solution sans volonté de baisser la qualité du modèle
  - analyse des résultats et retour sur le brief de Douglas

Le **choix de l'estimateur s'est porté sur SVR** pour ses résultats, sa stabilité dans les résultats (résistance aux valeurs atypiques et bruits selon les batches aléatoires) et sa rapidité d'entraînement (25 fois plus rapide que GradBoostRegressor et presque 80 fois plus rapide que RdmForestRegressor).

L'**hyperparamétrage** de la régression à support de vecteur pouvant réduire le sur-apprentissage, il a été testé comme suit :
- maintien du noyau linéaire
- hyperparamètre C sur 10 valeurs entre 0.1 et 1.0 (il gère la régularisation du modèle, sa capacité à pondérer le bruit dans les observations et à généraliser)
- constat : variations des optimums selon l'échantillonnage
- utilisation d'une validation croisée à 5 blocs (`sklearn.model_selection.GridSearchCV`) pour lisser les résultats
- valeur optimale de C : 1.0 (celle de départ)

Enfin, deux derniers tests ont été effectués :
- impact de l'ENERGYSTARScore avec une  : aucun changement dans les trois métriques après suppression de la variable en amont du traitement : **l'ENERGYSTAR Score n'a pas d'influence sur ce modèle**
- **impact du ratio d'usage non résidentiel des bâtiments** vu dans l'EDA : évolution remarquable bien que située entre deux valeurs très proches, un impact plus important avec un autre modèle ou d'autres données est susceptible de se produire

La **très forte corrélation entre la variable structurelle définissant une consommation de gaz naturel par pied carré et la cible** (émissions de gaz à effet de serre) fausse le modèle et lui fait obtenir des résultats prédictifs presque parfaits.   
Pour être certain de **se prémunir contre une fuite de données qui entraînerait la nécessaire refonte de cette étude**, il est important de **vérifier** avec Douglas le project lead **que nous pouvons bien obtenir un relevé de consommation de gaz la première année** pour créer cette variable structurelle.

D'autres **points d'améliorations à aborder avec Douglas**, le lead project, sont listés en conclusion comme les précisions l'usage non résidentiel ou le manque de fiabilité des données concernant les parkings.

# <a id='toc2_'></a>[Introduction](#toc0_)

## <a id='toc2_1_'></a>[Prérequis et imports](#toc0_)

Pré-requis de fonctionnement :

Packages utilisés :

```
Python 3.9.18
-----
matplotlib          3.7.2
IPython             8.15.0
jupyter_client      7.4.9
jupyter_core        5.3.0
jupyterlab          3.6.3
notebook            6.5.4
numpy               1.25.2
pandas              2.0.3
plotly              5.9.0
scipy               1.11.1
seaborn             0.12.2
session_info        1.0.0
sklearn             1.3.0
```

In [1]:
import logging
import pickle
import time
from functools import partial
import warnings

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.inspection import permutation_importance

# render in GitHub & NBViewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# prevent warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# logging configuration (see all outputs, even DEBUG or INFO)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## <a id='toc2_2_'></a>[Fonctions spécifiques](#toc0_)

In [2]:
def nan_warn(df, nan_col="nan_pct", tags_col="tags", thresh=0.4):
    """
    Warns if NaNs outpass a defined threshold,
    warning showed as a tag in the dataframe on a defined column.

    Inputs:
    • df: dataframe
    • nan_col: dataframe column (string, default = "nan")
    • tags_col: dataframe column (string, default = "tags")
    • thresh: threshold for NaNs warning (float, default = 0.4)

    Output: modified dataframe

    Requirements: pandas
    """

    df_ = df.copy()
    mask = df_[nan_col] / 100 >= thresh
    df_.loc[mask, tags_col] = df_.loc[mask, tags_col] + "🚫"

    return df_

def type_tag(df, uni_col="unique", type_col="type", count_col="count",
        tags_col="tags"):
    """
    Defines a type tag of a dataframe feature,
    depending on unique values, count and dtype,
    and writes it in a tag column.

    Inputs:
    • df: dataframe
    • uni_col: dataframe column (string, default = "unique")
    • type_col: dataframe column (string, default = "type")
    • count_col: dataframe column (string, default = "type")
    • tags_col: dataframe column (string, default = "tags")

    Output: modified dataframe

    Requirements: pandas
    """

    df_ = df.copy()
    total_count = max(df_[count_col])

    # const warn
    const_mask = df_[uni_col] == 1
    df_.loc[const_mask, tags_col] = df_.loc[const_mask, tags_col] + "🔒"

    # unique warn
    uniq_mask = df_[uni_col] == total_count
    df_.loc[uniq_mask, tags_col] = df_.loc[uniq_mask, tags_col] + "💎"

    # bool = categorical feat
    is_bool_mask = df_[type_col] == "bool"
    df_.loc[is_bool_mask, tags_col] = df_.loc[is_bool_mask, tags_col] + "📦"
    
    # object categorical feat
    type_mask = df_[type_col] == "object"
    # define limit
    categ_limit = int(max(2, min(60, total_count / 1.2)))
    # filter
    categ_mask = df_[uni_col].between(2, categ_limit)
    df_.loc[(categ_mask & type_mask), tags_col] = df_.loc[
        (categ_mask & type_mask), tags_col] + "📦"

    return df_

def describe_df(df, nan_thresh=0.4):
    """
    Dataframe describer, include little more information than .describe()

    Inputs:
    • df: dataframe to be analysed
    • nan_thresh: threshold for NaNs warning (float, default = 0.4)

    Output: dataframe of data description

    Requirements: pandas, numpy
    """

    df_ = df.describe(include="all").T
    df_.sort_index(inplace=True)
    df_["unique"] = df.nunique()
    df_["type"] = df.dtypes
    df_["nan"] = df.isna().sum()
    df_["nan_pct"] = np.round(df.isna().mean()*100, 2)
    
    # tags column
    df_.insert(0, "tags", "")
    # nan warning tag
    df_ = nan_warn(df_, thresh=nan_thresh)
    # type check + const warn tag
    df_ = type_tag(df_,
        uni_col="unique",
        type_col="type",
        count_col="count",
        tags_col="tags",
        )
    
    df_ = df_.fillna("-")

    return df_

def impact_classif(value, thresh=30):
    """
    Returns an impact classification depending on a value
    and a threshold.

    Positional arguments: 
    -------------------------------------
    value: float or int: between 0 and 100

    Optional arguments: 
    -------------------------------------
    thresh: float or int: threshold to adjust the function, default=30

    Output: string, warning intensity (int)
    """

    if value == 0:
        return "⌀", False
    elif 0 < value < (thresh / 6):
        return "--", False
    elif (thresh / 6) <= value < (thresh / 3):
        return "-", False
    elif (thresh / 3) <= value < (thresh * 2 / 3):
        return "+", False
    elif (thresh * 2 / 3) <= value < thresh:
        return "++", False
    elif thresh <= value < (thresh + (thresh / 3)):
        return "⚠️", 1
    elif (thresh + (thresh / 3)) <= value < (thresh + (2 * thresh / 3)):
        return "⚠️⚠️", 2
    elif (thresh + (2 * thresh / 3)) <= value < 75:
        return "⚠️⚠️⚠️", 3
    elif value >= 75:
        return "☠️", 4
    else:
        return "❓", False

def impact(df_before, df_after, monitored=None):
    """
    Returns an impact dataframe from an original and
    a second dataframe.
    Impact is calculated on the columns and population,
    plus on some optional arguments.

    Positional arguments: 
    -------------------------------------
    df_before: dataframe: original dataframe (starting point, before action)
    df_before: dataframe: original dataframe (starting point, after action)

    Optional arguments: 
    -------------------------------------
    monitored: list of strings: Columns to check.
               ⚠️ Columns must be present in both dataframes.
               Default = None

    Output: dataframe in logging.info()

    Required modules: pandas, numpy, logging
    """

    pop_bef = df_before.shape[0]
    cols_bef = df_before.shape[1]
    
    pop_aft = df_after.shape[0]
    cols_aft = df_after.shape[1]

    diff_pop = pop_bef - pop_aft
    diff_cols = cols_bef - cols_aft

    prct_pop_num = np.round(diff_pop / pop_bef * 100, 2)
    prct_cols_num = np.round(diff_cols / cols_bef * 100, 2)
    prct_pop = f"{prct_pop_num}%"
    prct_cols = f"{prct_cols_num}%"

    imp_cols = impact_classif(prct_cols_num)
    imp_pop = impact_classif(prct_pop_num)

    # list of potential warnings
    warn_logs = [imp_cols[1], imp_pop[1]]

    _df_ = pd.DataFrame([
        [cols_bef, pop_bef],
        [cols_aft, pop_aft],
        [diff_cols, diff_pop],
        [prct_cols, prct_pop],
        [imp_cols[0], imp_pop[0]]],
        columns=["Columns", "Population"],
        index=["Before", "After", "Difference", "Prct", "IMPACT"]
        )

    if monitored:
        for m in monitored:
            # get output from command
            before_ = df_before[m].count()
            after_ = df_after[m].count()
            prct_ = np.round((before_ - after_) / before_ * 100, 2)
            imp = impact_classif(prct_)
            # add in DF
            _df_[m] = [before_,
                    after_,
                    before_ - after_,
                    f"{prct_}%",
                    imp[0]
                ]
            # add potential warning
            warn_logs.append(imp[1])

    if (1 in warn_logs or 2 in warn_logs or 3 in warn_logs):
        return logging.warning(display(_df_))
    elif 4 in warn_logs:
        return logging.critical(display(_df_))
    else:
        return logging.info(display(_df_))

def X_y_splitter(df, target, random_state=42, valid=False, verbose=False):
    """
    Splits a dataframe in random train, test and even validation samples,
    plus same for the target.

    Inputs:
    • df: original dataframe
    • target: targetted feature (string)
    • random_state: for randomization fixing (int)
    • valid: if a validation split is needed (bool, default = False)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: 4 or 6 random dataframe samples

    Requirements: pandas, sklearn, logging
    """

    X = df.copy()
    X.drop(target, axis=1, inplace=True)
    y = df[target]

    if verbose:
        logging.info(f"{X.shape = }, {y.shape = }")


    if valid:
        # train / test : 70-30%
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=random_state)

        # validation / test : 50-50% (= 15-15% of total data)
        X_val, X_test, y_val, y_test = train_test_split(
            X_test, y_test, test_size=0.5, random_state=random_state)

        if verbose:
            logging.info(f"{X_train.shape = }, {y_train.shape = }\n" +
                f"{X_val.shape = }, {y_val.shape = }\n" +
                f"{X_test.shape = }, {y_test.shape = }")

        return X_train, X_val, X_test, y_train, y_val, y_test

    # train / test : 80-20%
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state)

    if verbose:
        logging.info(f"{X_train.shape = }, {y_train.shape = }\n" +
            f"{X_test.shape = }, {y_test.shape = }")

    return X_train, X_test, y_train, y_test

def X_splitter(df, random_state=42, valid=False, verbose=False):
    """
    Splits a dataframe in random train, test and even validation samples.

    Inputs:
    • df: original dataframe
    • random_state: for randomization fixing (int)
    • valid: if a validation split is needed (bool, default = False)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: 2 or 3 random dataframe samples

    Requirements: pandas, sklearn, logging
    """

    if valid:
        # train / test : 70-30%
        X_train, X_test = train_test_split(
            df, test_size=0.3, random_state=random_state)

        # validation / test : 50-50% (= 15-15% of total data)
        X_val, X_test = train_test_split(
            X_test, test_size=0.5, random_state=random_state)

        if verbose:
            logging.info(f"{df.shape = }, {X_train.shape = }\n" +
                f"{X_val.shape = }, {X_test.shape = }")

        return X_train, X_val, X_test

    # train / test : 80-20%
    X_train, X_test = train_test_split(
        df, test_size=0.2, random_state=random_state)

    if verbose:
        logging.info(f"{df.shape = }, {X_train.shape = }, {X_test.shape = }")

    return X_train, X_test

def y_splitter(df, target, verbose=False):
    """
    Separates a dataframe from its target.

    Inputs:
    • df: original dataframe
    • target: targetted feature (string)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: 1 dataframe and 1 series (target)

    Requirements: pandas, sklearn, logging
    """

    X = df.copy()
    X.drop(target, axis=1, inplace=True)
    y = df[target]

    if verbose:
        logging.info(f"{X.shape = }, {y.shape = }")

    return X, y

def compare_splits(split1, split2, name1, name2, add_name=None, plot=False):
    """
    Compare quickly 2 target Series splits.

    Inputs:
    • y_train, y_test: 2 Pandas series
    • name1, name2: 2 names (str)
    • name: name to append to compared series (str, default = None)
    • plot: displays comparative boxplot (bool, default = False)

    Output: comparison dataframe (and even boxplot)

    Requirements: numpy, pandas, plotly
    """
    
    add_name = str(add_name) if add_name is not None else ""

    compere = pd.DataFrame(columns=["pop", "min", "max", "mean", "med", "std"])
    
    compere.loc[name1 + add_name] = [split1.shape[0], split1.min(),
        split1.max(), split1.mean(), split1.median(), split1.std()]
    
    compere.loc[name2 + add_name] = [split2.shape[0], split2.min(),
        split2.max(), split2.mean(), split2.median(), split2.std()]

    if plot:
        fig = go.Figure()
        fig.add_trace(go.Box(x=split1, name=name1, boxmean='sd'))
        fig.add_trace(go.Box(x=split2, name=name2, boxmean='sd'))
        fig.update_layout(height=400, width=1200, yaxis=dict(automargin=True),
            title="Splits comparison")

        fig.show()
    
    return compere

def display_r2(action):
    """
    Display r² graphs for a given action (stored in pickle object).

    Input: action (element of pickle object data_leak_scores.p)

    Requirements: numpy, pandas, plotly
    """
    
    print(action[0])

    scores = action[2]

    # r² results
    r2_all = scores.loc[scores["metric"] == "r2"]
    # global mean score for a same random state
    r2_all.loc["TOTAL"] = [
        "-",
        "-",
        round(r2_all["LinReg"].mean(), 3),
        round(r2_all["SVR"].mean(), 3),
        round(r2_all["RdmForestReg"].mean(), 3),
        round(r2_all["GradBoostReg"].mean(), 3),
        round(r2_all["MEAN"].mean(), 3),
        round(r2_all["MEDIAN"].mean(), 3),
        ]
    display(r2_all)

    # graphic figure
    df_ = r2_all.drop("TOTAL", axis=0)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["LinReg"],
        mode='lines+markers', name='LinReg'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["SVR"],
        mode='lines+markers', name='SVR'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["RdmForestReg"],
        mode='lines+markers', name='RdmForestReg'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["GradBoostReg"],
        mode='lines+markers', name='GradBoostReg'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["MEAN"],
        mode='lines+markers', name='MEAN'))
    fig.show()

    return

def display_all_r2(scores_list):
    """
    Displays a plot of all r² scores for step by step analysis.
    """

    columns = ['LinReg', 'SVR', 'RdmForestReg', 'GradBoostReg', "mean",
        "median"]
    full_r2 = pd.DataFrame(columns=columns)

    for f in scores_list:
        # get variable name
        name = f[0]
        
        _ = f[3].loc[f[3]["metric"] == "r2"]
        linreg = round(_["LinReg"].mean(), 3)
        svr = round(_["SVR"].mean(), 3)
        rdmforest = round(_["RdmForestReg"].mean(), 3)
        gradboost = round(_["GradBoostReg"].mean(), 3)

        full_r2.loc[name] = [
            linreg,
            svr,
            rdmforest,
            gradboost,
            round(np.mean([linreg, svr, rdmforest, gradboost]), 3),
            round(np.median([linreg, svr, rdmforest, gradboost]), 3),
        ]

        # mask if too high / low
        full_r2.mask(full_r2 > 3, 3, inplace=True)
        full_r2.mask(full_r2 < -3, -3, inplace=True)
 
    # graphic figure
    layout = go.Layout(yaxis=dict(range=[-0.5, 1]))
    fig = go.Figure(layout=layout)
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["LinReg"],
        mode='lines+markers', name='LinReg'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["SVR"],
        mode='lines+markers', name='SVR'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["RdmForestReg"],
        mode='lines+markers', name='RdmForestReg'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["GradBoostReg"],
        mode='lines+markers', name='GradBoostReg'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["mean"],
        mode='lines+markers', name='mean'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["median"],
        mode='lines+markers', name='median'))
    fig.show()

    print("""save_actions = BASE: drop duplicates (0), keep compliants only, clean neighborhood, drop useless feats, drop nans except ENERGYSTARScore, sort columns
save_actions_2    = BASE + filter GHGEmissionsIntensity, NumberofBuildings, NumberofFloors, PropertyGFATotal (22)
save_actions_3    = ........ + create BuildingRatio feature
save_actions_4    = ............ + drop PropertyGFABuilding(s) feature
save_actions_5    = ................ + filter BuildingRatio (19)
save_actions_6    = .................|.. + create ParkingRatio feature
save_actions_6_1  = .................|...... + drop PropertyGFAParking feature
save_actions_6_2  = .................|.......... + filter ParkingRatio (8)
save_actions_7    = ................ + create AreaPerFloor(sf) and AreaPerBldg(sf) features
save_actions_7_1  = .................... + drop NumberofFloors and NumberofBuildings features
save_actions_7_2  = ........................ + filter AreaPerFloor(sf) (1)
save_actions_8    = ............................ + create SteamUse_I(kBtu/sf), Electricity_I(kBtu/sf) and NaturalGas_I(kBtu/sf) features
save_actions_8_1  = .............................|.. + drop SteamUse(kBtu), Electricity(kBtu) and NaturalGas(kBtu) features
save_actions_8_2  = .............................|...... + filter SteamUse_I(kBtu/sf) (6), Electricity_I(kBtu/sf) (6)
save_actions_9    = .............................|.......... + create NonResidentialRatio feature
save_actions_9_1  = .............................|.............. + filter NonResidentialRatio >= min_nr_pct (often > 1000)
save_actions_10   = .............................|.................. + create ParkingRatio, drop PropertyGFAParking, filter ParkingRatio (8)
save_actions_11   = ............................ + create NonResidentialRatio + filter >= min_nr_pct (often > 1000)
save_actions_11_1 = .............................|.. + create ParkingRatio, drop PropertyGFAParking feature, filter ParkingRatio (8)
save_actions_12   = ............................ + STEAMUSE: create SteamUse_I(kBtu/sf), drop SteamUse(kBtu), filter SteamUse_I(kBtu/sf) (6)
save_actions_13   = ............................ + ELECTRICITY: create Electricity_I(kBtu/sf), drop Electricity(kBtu), filter Electricity_I(kBtu/sf) (6)
save_actions_14   = ............................ + NATURALGAS: create NaturalGas_I(kBtu/sf), drop NaturalGas(kBtu)
""")

    return

def permut_fi(process_result):
    """
    Plots feature importance using permutation techinque for all estimators
    of a given process result.

    Input: result of the process_all() function.

    Requirements: pandas, numpy, sklearn, plotly
    """

    pipes = process_result["pipes"]
    # delete "Dummy" if exists
    pipes.pop("Dummy", None)
    X_train = process_result["splits"]["X_train"]
    y_train = process_result["splits"]["y_train"]

    # preparing plot
    fig = make_subplots(rows=(len(pipes)), cols=1)

    # loop over estimators
    for i, e in enumerate(pipes):
        # perform permutation importance
        fi = permutation_importance(pipes[e], X_train, y_train,
            scoring='neg_mean_squared_error')
        # get importance (mean)
        importance = fi.importances_mean

        # plot it
        fig.add_trace(go.Bar(x=X_train.columns, y=importance,
            # text=importance, textposition='auto',
            name=e), row=i+1, col=1)

    # display adjustments
    fig.update_layout(title="Feature Importance",
        width=800, height=len(pipes)*200)
    fig.update_traces(textposition="outside", cliponaxis=False)
    fig.update_xaxes(tickangle=60)
    # add vertical lines
    for col in range(1, X_train.shape[1]):
        fig.add_vline(x=col+0.5, line_width=1,
            line_dash="dash", line_color="grey")
    # display x-axis values for last axis only
    for r in range(1, i+1):
        fig.update_xaxes(title_text='', tickvals=[], row=r, col=1)

    fig.show()

    return


## <a id='toc2_3_'></a>[Fonctions de traitement](#toc0_)

In [3]:
def data_cleaner(df, min_nr_pct=0.0001, verbose=False):
    """

    Data cleaning, grouping explicitly all previously used cleaning methods.

    Inputs:
    • df: dataframe to clean
    • min_nr_pct: minimal non residential percentage (float, default = 0.0001)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: cleansed dataframe

    Requirements: numpy, pandas, logging
    """

    # DROP DUPLICATES
    # *************************************************************************
    df_ = df.drop_duplicates()

    # KEEP COMPLIANT DATA ONLY
    # *************************************************************************
    df_ = df_.loc[df_["ComplianceStatus"] == "Compliant"]

    # CLEAN NEIGHBORHOOD
    # *************************************************************************
    # case harmonization
    df_["Neighborhood"] = df_["Neighborhood"].str.upper()
    # duplicate deletion
    df_.loc[df_["Neighborhood"] == "DELRIDGE NEIGHBORHOODS",
        "Neighborhood"] = "DELRIDGE"
    
    # DELETE USELESS FEATURES
    # *************************************************************************
    df_.drop([
        "Address",
        "City",
        "Comments",
        "ComplianceStatus",
        "CouncilDistrictCode",
        "DataYear",
        "DefaultData",
        # 'Electricity(kBtu)',
        "Electricity(kWh)",
        # "ENERGYSTARScore",
        # 'GHGEmissionsIntensity',
        "Latitude",
        "ListOfAllPropertyUseTypes",
        "Longitude",
        # 'NaturalGas(kBtu)',
        "NaturalGas(therms)",
        # "Neighborhood",
        # "NumberofBuildings",
        # "NumberofFloors",
        "OSEBuildingID",
        "Outlier",
        # 'PropertyGFABuilding(s)',
        # 'PropertyGFAParking',
        # 'PropertyGFATotal',
        "PropertyName",
        "SiteEnergyUse(kBtu)",
        "SiteEnergyUseWN(kBtu)",
        "SiteEUI(kBtu/sf)",
        # 'SiteEUIWN(kBtu/sf)',
        "SourceEUI(kBtu/sf)",
        "SourceEUIWN(kBtu/sf)",
        "State",
        # 'SteamUse(kBtu)',
        "TaxParcelIdentificationNumber",
        "TotalGHGEmissions",
        # 'YearBuilt',
        "YearsENERGYSTARCertified",
        "ZipCode",
        ], axis=1, inplace=True)
    
    # BUILDING RATIO
    # *************************************************************************
    df_["BuildingRatio"] = df_["PropertyGFABuilding(s)"]\
        / df_["PropertyGFATotal"]
    
    # PARKING RATIO
    # *************************************************************************
    df_["pkg_gfa"] = 0
    # seek parking GFA and add up
    cols = ["ThirdLargestPropertyUseType", "SecondLargestPropertyUseType",
        "LargestPropertyUseType"]
    for c in cols:
        gfa = c + "GFA"
        is_pkg = df_[c].str.lower().str.contains(r'parking', na=False)
        df_[gfa].where(is_pkg, 0, inplace=True)
        # add GFA to total
        df_["pkg_gfa"] += df_[gfa]
    # keep highest GFA
    df_.loc[df_["pkg_gfa"] > df_["PropertyGFAParking"],
        "PropertyGFAParking"] = df_["pkg_gfa"]
    # apply % on total GFA
    df_["ParkingRatio"] = df_["PropertyGFAParking"] / df_["PropertyGFATotal"]
    df_.drop(["pkg_gfa"], axis=1, inplace=True)

    # NON-RESIDENTIAL RATIO (ALL USAGE FEATS COMPILATION)
    # *************************************************************************
    df_["non_res_gfa"] = 0
    lput_notna = df_["LargestPropertyUseType"] != np.NaN
    cols = ["ThirdLargestPropertyUseType", "SecondLargestPropertyUseType",
        "LargestPropertyUseType"
    ]
    # LargestPropertyUseType != NaN
    for c in cols:
        gfa = c + "GFA"
        is_res = df_[c].str.lower().str.contains(
            r'(?<!(non))(residential|multifamily|residence)', na=True)
        df_[gfa].mask(lput_notna & is_res, 0, inplace=True)
        # add GFA to total
        df_["non_res_gfa"] += df_[gfa]
    # LargestPropertyUseType == NaN
    zero_non_res_gfa = df_["non_res_gfa"] != 0
    is_res = df_["PrimaryPropertyType"].str.lower().str.contains(
        r'(?<!(non))(residential|multifamily|residence)', na=True)
    df_["non_res_gfa"].where(lput_notna & is_res | zero_non_res_gfa,
        df_["PropertyGFATotal"], inplace=True)
    # apply % on total GFA
    df_["NonResidentialRatio"] = df_["non_res_gfa"] / df_["PropertyGFATotal"]
    df_.loc[df_["NonResidentialRatio"] > 1, "NonResidentialRatio"] = 1
    df_.drop(["non_res_gfa"], axis=1, inplace=True)

    # DROP POPULATION UNDER A NON-RESIDENTIAL RATIO
    # *************************************************************************
    df_ = df_.loc[(df_["NonResidentialRatio"] >= min_nr_pct)]

    # CHANGE RAW ENERGY VALUES TO ENERGY INTENSITY
    # *************************************************************************
    df_["SteamUse_I(kBtu/sf)"] = df_["SteamUse(kBtu)"]\
        / df_["PropertyGFATotal"]
    df_["Electricity_I(kBtu/sf)"] = df_["Electricity(kBtu)"]\
        / df_["PropertyGFATotal"]
    df_["NaturalGas_I(kBtu/sf)"] = df_["NaturalGas(kBtu)"]\
        / df_["PropertyGFATotal"]
        
    # SET MINIMUM NUMBER OF FLOORS TO 1 AND ADD AREA PER FLOOR FEATURE
    # *************************************************************************
    df_.loc[(df_["NumberofFloors"] == 0) & (df_["BuildingRatio"] > 0),
        "NumberofFloors"] = 1
    df_["AreaPerFloor(sf)"] = df_["NumberofFloors"]\
        / df_["PropertyGFATotal"]

    # SET MINIMUM NUMBER OF BUILDINGS TO 1 AND ADD AREA PER BUILDING FEATURE
    # *************************************************************************
    df_.loc[(df_["NumberofBuildings"] == 0) & (df_["BuildingRatio"] > 0),
        "NumberofBuildings"] = 1
    df_["AreaPerBldg(sf)"] = df_["NumberofBuildings"]\
        / df_["PropertyGFATotal"]

    # CHASE STATISTICAL OUTLIERS
    # *************************************************************************
    df_ = df_.loc[df_["GHGEmissionsIntensity"] <= 20] # 2 individuals
    df_ = df_.loc[df_["AreaPerFloor(sf)"] <= 0.004] # 1 individual
    df_ = df_.loc[df_["BuildingRatio"] >= 0.4] # 19 individuals
    df_ = df_.loc[df_["NumberofBuildings"] <= 20] # 3 individuals
    df_ = df_.loc[df_["NumberofFloors"] <= 40] # 16 individuals
    df_ = df_.loc[df_["ParkingRatio"] <= 0.8] # 8 individuals
    df_ = df_.loc[df_["PropertyGFATotal"] <= 2000000] # 1 individual
    df_ = df_.loc[df_["SteamUse_I(kBtu/sf)"] <= 100] # 6 individuals
    df_ = df_.loc[df_["Electricity_I(kBtu/sf)"] <= 350] # 6 individuals

    # DELETE LAST USELESS FEATURES
    # *************************************************************************
    df_.drop([
        "BuildingType",
        "PrimaryPropertyType",
        "LargestPropertyUseType",
        "LargestPropertyUseTypeGFA",
        "SecondLargestPropertyUseType",
        "SecondLargestPropertyUseTypeGFA",
        "ThirdLargestPropertyUseType",
        "ThirdLargestPropertyUseTypeGFA",
        "PropertyGFABuilding(s)", # => BuildingRatio
        "PropertyGFAParking", # => ParkingRatio
        "SteamUse(kBtu)",
        "Electricity(kBtu)",
        "NaturalGas(kBtu)",
        "NumberofFloors",
        "NumberofBuildings",
    ], axis=1, inplace=True)

    # DROP NANS EXCEPT FOR ENERGY STAR SCORE FEATURE
    # *************************************************************************
    feats = ["ENERGYSTARScore"]
    df_.dropna(subset=df_.columns.difference(feats), inplace=True)

    # SORT COLUMNS
    # *************************************************************************
    df_.sort_index(axis=1, inplace=True)

    # SHOW GLOBAL IMPACT
    if verbose:
        logging.info("""\n***************************************************
    👇👇   GLOBAL IMPACT  👇👇""")
        impact(df, df_)

    return df_

def process_all(df, non_res_min_usage, random_state, target, scores_df,
        verbose=False, plot=False):
    """
    All-in-one processing, for parameters comparisons.
    
    Inputs:
    • df: dataframe to process
    • non_res_min_usage: minimal non residential percentage (float)
    • random_state: (x,y) split random seed (int)
    • target: targetted feature, must be in df (string)
    • scores_df: dataframe (empty or not) structured as follows:
        scores_df = pd.DataFrame(columns=["rdm_st", "metric", "LinReg", "SVR",
            "RdmForestReg", "GradBoostReg", "MEAN", "MEDIAN"])
    • verbose: determines whether logs are shown or not (bool, default = False)
    • plot: plots results or not (bool, default = False)

    Output:
    • dict containing :
        - target splits dataframes
        - trained estimators
        - updated scores_df dataframe

    Requirements: numpy, pandas, sklearn, logging, plotly
    """

    df_ = df.copy()
    process = {}

    # PARAMS
    # *************************************************************************
    non_res_min_usage = non_res_min_usage
    random_state = random_state
    target = target
    classif_cols = ["Neighborhood"]

    # CLEAN
    # *************************************************************************
    df_ = data_cleaner(df_, non_res_min_usage)

    # REMAINING PARAMS (must be done after cleaning)
    # *************************************************************************
    num_cols = df_.drop(classif_cols, axis=1).columns.to_list()
    num_cols.remove(target)

    # PIPELINES DEFINITIONS
    # *************************************************************************
    col_transf = ColumnTransformer(
        [
            ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
            ('min_max_scaler', MinMaxScaler(), num_cols),
        ],
        remainder = 'passthrough',
        verbose_feature_names_out=False
    )

    imputer = Pipeline(
        [
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    preprocessor = Pipeline(
        [
            ("col_transf", col_transf),
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    # SPLIT
    # *************************************************************************
    X_train, X_test, y_train, y_test = X_y_splitter(
        df_, target, random_state)
    
    # -> to dataframe
    X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
            columns=preprocessor[0:].get_feature_names_out())
    X_test = pd.DataFrame(preprocessor.fit_transform(X_test),
            columns=preprocessor[0:].get_feature_names_out())
    
    # for output
    process["splits"] = {"X_train": X_train, "X_test": X_test,
        "y_train": y_train, "y_test": y_test}

    # ESTIMATORS
    # *************************************************************************
    models_names = ["Dummy", "LinReg", "SVR", "RdmForestReg", "GradBoostReg"]
    
    # pipelines
    pipelines = [
        Pipeline([('Dummy', DummyRegressor(strategy="mean"))]),
        Pipeline([('LinReg', LinearRegression())]),
        Pipeline([('SVR', svm.SVR(kernel='linear'))]),
        Pipeline([('RdmForestReg', RandomForestRegressor())]),
        Pipeline([('GradBoostReg', GradientBoostingRegressor())]),
    ]

    # scores (DF preparation)
    row_name = "rdm_st_" + str(random_state)
    scores_df.loc[row_name + "_r2"] = "-"
    scores_df.loc[row_name + "_mae"] = "-"
    scores_df.loc[row_name + "_rmse"] = "-"
    scores_df.loc[row_name + "_ttime"] = "-"

    scores_df["rdm_st"][row_name + "_r2"] = random_state
    scores_df["metric"][row_name + "_r2"] = "r2"

    scores_df["rdm_st"][row_name + "_mae"] = random_state
    scores_df["metric"][row_name + "_mae"] = "mae"

    scores_df["rdm_st"][row_name + "_rmse"] = random_state
    scores_df["metric"][row_name + "_rmse"] = "rmse"
    
    scores_df["rdm_st"][row_name + "_ttime"] = random_state
    scores_df["metric"][row_name + "_ttime"] = "seconds"

    # loop over estimators
    # for output
    process["pipes"] = {}
    for p, name in zip(pipelines, models_names):
        start = time.time()
        p.fit(X_train, y_train)
        end = time.time()
        fit_time = end - start

        r2 = round(r2_score(y_test, p.predict(X_test)), 3)
        mae = round(mean_absolute_error(y_test, p.predict(X_test)), 3)
        rmse = round(mean_squared_error(y_test, p.predict(X_test),
            squared = False), 3)

        # save scores in DF
        if name != "Dummy":
            scores_df[name][row_name + "_r2"] = r2
            scores_df[name][row_name + "_mae"] = mae
            scores_df[name][row_name + "_rmse"] = rmse
            scores_df[name][row_name + "_ttime"] = fit_time

        # for output
        process["pipes"][name] = p

    # means and medians for estimators
    scores_df["MEAN"][row_name + "_r2"] = round(
        scores_df.loc[row_name + "_r2"][2:6].mean(), 3)
    scores_df["MEAN"][row_name + "_mae"] = round(
        scores_df.loc[row_name + "_mae"][2:6].mean(), 3)
    scores_df["MEAN"][row_name + "_rmse"] = round(
        scores_df.loc[row_name + "_rmse"][2:6].mean(), 3)

    scores_df["MEDIAN"][row_name + "_r2"] = round(
        scores_df.loc[row_name + "_r2"][2:6].median(), 3)
    scores_df["MEDIAN"][row_name + "_mae"] = round(
        scores_df.loc[row_name + "_mae"][2:6].median(), 3)
    scores_df["MEDIAN"][row_name + "_rmse"] = round(
        scores_df.loc[row_name + "_rmse"][2:6].median(), 3)

    # for output
    process["scores"] = scores_df

    # scores
    if plot:
        # show r² on a plot
        scor_plt = scores_df.iloc[0,2:]
        x = models_names[1:] # define x

        fig = go.Figure()
        fig.add_trace(go.Bar(x=x, y=scor_plt[:4], name="r² scores",
            text=scor_plt[:4]))

        # mean horizontal line
        fig.add_trace(go.Scatter(x=x, y=[scor_plt[4]]*len(x),
            mode='lines', line_dash="dash",
            line=dict(color='red', width=3),
            name=f"mean={scor_plt[4]}"))
        # median horizontal line
        fig.add_trace(go.Scatter(x=x, y=[scor_plt[5]]*len(x),
            mode='lines', line_dash="dash",
            line=dict(color='black', width=3),
            name=f"median={scor_plt[5]}"))

        fig.update_layout(title=f"r² for random state = {random_state}",
            width=700, height=300, showlegend=True)
        fig.show()
    if verbose:
        display(scores_df)

    # compare splits
    splits_comp = compare_splits(y_train, y_test, "y_train", "y_test",
        plot=plot)
    if verbose:
        display(splits_comp)

    return process

def process_svr(df, non_res_min_usage, random_state, target, scores_df,
        C=1.0, verbose=False, plot=False):
    """
    All-in-one processing, for SCR hyperparameters comparisons.
    
    Inputs:
    • df: dataframe to process
    • non_res_min_usage: minimal non residential percentage (float)
    • random_state: (x,y) split random seed (int)
    • target: targetted feature, must be in df (string)
    • scores_df: dataframe (empty or not) structured as follows:
        scores_df = pd.DataFrame(columns=["rdm_st", "metric", "LinReg", "SVR",
            "RdmForestReg", "GradBoostReg", "MEAN", "MEDIAN"])
    • C: SVR C hyperparameter (float between ) and 1, default = 1.0)
    • verbose: determines whether logs are shown or not (bool, default = False)
    • plot: plots results or not (bool, default = False)

    Output:
    • dict containing :
        - target splits dataframes
        - trained estimators
        - updated scores_df dataframe

    Requirements: numpy, pandas, sklearn, logging, plotly
    """

    df_ = df.copy()
    process = {}

    # PARAMS
    # *************************************************************************
    non_res_min_usage = non_res_min_usage
    random_state = random_state
    target = target
    classif_cols = ["Neighborhood"]

    # CLEAN
    # *************************************************************************
    df_ = data_cleaner(df_, non_res_min_usage)

    # REMAINING PARAMS (must be done after cleaning)
    # *************************************************************************
    num_cols = df_.drop(classif_cols, axis=1).columns.to_list()
    num_cols.remove(target)

    # PIPELINES DEFINITIONS
    # *************************************************************************
    col_transf = ColumnTransformer([
            ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
            ('min_max_scaler', MinMaxScaler(), num_cols),
            ],
        remainder = 'passthrough',
        verbose_feature_names_out=False
        )

    imputer = Pipeline([
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ])

    preprocessor = Pipeline([
            ("col_transf", col_transf),
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ])

    # SPLIT
    # *************************************************************************
    X_train, X_test, y_train, y_test = X_y_splitter(
        df_, target, random_state)
    
    # -> to dataframe
    X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
        columns=preprocessor[0:].get_feature_names_out())
    X_test = pd.DataFrame(preprocessor.fit_transform(X_test),
        columns=preprocessor[0:].get_feature_names_out())
    
    # for output
    process["splits"] = {"X_train": X_train, "X_test": X_test,
        "y_train": y_train, "y_test": y_test}

    # SVR ESTIMATOR
    # *************************************************************************
    name = "SVR"
    svr_pipe = Pipeline([('SVR', svm.SVR(kernel='linear', C=C))])

    # scores (DF preparation)
    row_name = "SVR_rs_" + str(random_state) + "_C_" + str(C)
    scores_df.loc[row_name + "_r2"] = "-"
    scores_df.loc[row_name + "_mae"] = "-"
    scores_df.loc[row_name + "_rmse"] = "-"
    scores_df.loc[row_name + "_ttime"] = "-"

    scores_df["rdm_st"][row_name + "_r2"] = random_state
    scores_df["metric"][row_name + "_r2"] = "r2"

    scores_df["rdm_st"][row_name + "_mae"] = random_state
    scores_df["metric"][row_name + "_mae"] = "mae"

    scores_df["rdm_st"][row_name + "_rmse"] = random_state
    scores_df["metric"][row_name + "_rmse"] = "rmse"
    
    scores_df["rdm_st"][row_name + "_ttime"] = random_state
    scores_df["metric"][row_name + "_ttime"] = "seconds"

    # for output
    process["pipes"] = {}

    start = time.time()
    svr_pipe.fit(X_train, y_train)
    end = time.time()
    fit_time = end - start

    # set scores
    prediction = svr_pipe.predict(X_test)
    r2 = round(r2_score(y_test, prediction), 3)
    mae = round(mean_absolute_error(y_test, prediction), 3)
    rmse = round(mean_squared_error(y_test, prediction, squared = False), 3)

    # save scores in DF
    scores_df[name][row_name + "_r2"] = r2
    scores_df[name][row_name + "_mae"] = mae
    scores_df[name][row_name + "_rmse"] = rmse
    scores_df[name][row_name + "_ttime"] = fit_time

    # for output
    process["pipes"][name] = svr_pipe
    process["scores"] = scores_df

    if verbose:
        display(scores_df)

    # compare splits
    splits_comp = compare_splits(y_train, y_test, "y_train", "y_test",
        plot=plot)
    if verbose:
        display(splits_comp)

    return process

def plot_svr_c_scores(rdm_state=42, verbose=False):
    """
    Plot SVR C hyperparameter tests scores, depending on a given random state.

    Input:
    • rdm_state (int, default = 42)
    • verbose: determines whether logs are shown or not (bool, default = False)
    
    Output: scores dataframe

    Requirements: numpy, pandas, sklearn, plotly
    """

    SVR_C_scores = pd.DataFrame(columns=["C", "r²", "MAE", "RMSE"])
    C_tests = (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1)

    for C in C_tests:
        skor = scores_df_model.copy()
        r = "rs_" + str(rdm_state) + "_C_" + str(C)
        ml = process_svr(data_raw, non_res_min_usage, rdm_state, target, skor,
            C=C, verbose=verbose)
        SVR_C_scores.loc[r] = [C, ml["scores"]["SVR"][0], ml["scores"]["SVR"][1],
            ml["scores"]["SVR"][2]]

    # plot it
    fig = go.Figure()
    x = C_tests
    fig.add_trace(go.Scatter(x=x, y=SVR_C_scores["r²"], name="r²"))
    fig.add_trace(go.Scatter(x=x, y=SVR_C_scores["MAE"], name="MAE"))
    fig.add_trace(go.Scatter(x=x, y=SVR_C_scores["RMSE"], name="RMSE"))
    fig.update_xaxes(autorange="reversed")
    fig.update_layout(width=600, height=400,
        title=f"Scores by SVR's C param. for RdmSt = {rdm_state}")
    fig.show()

    return SVR_C_scores

def get_best_svr_cv(df, non_res_min_usage, target, ESS=True, cv=5, plot=False):
    """
    All-in-one processing for SCR cross validation tests.
    
    Inputs:
    • df: dataframe to process
    • non_res_min_usage: minimal non residential percentage (float)
    • target: targetted feature, must be in df (string)
    • ESS: if False, drops the ENERGYSTARScore feature (bool, default = True)
    • cv: number of folds in the K-Fold cross validation (int, default = 5)
    • plot: plots results or not (bool, default = False)

    Output (dict):
    • df: preprocessed dataframe
    • X: df with only features, target dropped
    • y: df target only
    • gs: object with all GridSearch cross validation results

    Requirements: numpy, pandas, sklearn, logging, plotly
    """

    # SET PARAMS
    # *************************************************************************
    df_ = df.copy()
    output = {}
    # set list of 10 C hyperparameter values from 0.1 to 1.0 included
    Cs = [round(n, 1) for n in [*np.arange(0.1, 1.1, 0.1)]]
    # set tested parameters
    param_grid = {
        'SVR__kernel': ['linear'],
        'SVR__C': Cs,
        }

    # CLEAN
    # *************************************************************************
    df_ = data_cleaner(df_, non_res_min_usage)
    # drop ENERGYSTARScore if asked
    if not ESS:
        df_.drop("ENERGYSTARScore", axis=1, inplace=True)

    # SEPARATE NUMERIC / CLASSIF FEATURES
    # *************************************************************************
    classif_cols = ["Neighborhood"]
    num_cols = df_.drop(classif_cols, axis=1).columns.to_list()
    num_cols.remove(target)

    # PREPROCESSING PIPELINES
    # *************************************************************************
    col_transf = ColumnTransformer([
            ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
            ('min_max_scaler', MinMaxScaler(), num_cols),
            ],
        remainder = 'passthrough',
        verbose_feature_names_out=False
        )

    imputer = Pipeline([
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ])

    preprocessor = Pipeline([
            ("col_transf", col_transf),
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ])

    # apply preprocessing
    df_ = pd.DataFrame(preprocessor.fit_transform(df_),
        columns=preprocessor[0:].get_feature_names_out())

    # X, y split
    y = df_[target]
    X = df_.drop(target, axis=1)

    # output data
    output["df"] = df_
    output["X"] = X
    output["y"] = y

    # SVR ESTIMATOR & GRID SEARCH CV
    # *************************************************************************
    svr_pipe = Pipeline([('SVR', svm.SVR())])
    scoring = ["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"]
    gscv = GridSearchCV(svr_pipe, param_grid, cv=cv, n_jobs=-1,
        scoring=scoring, refit="r2")
    gscv.fit(X, y)

    # RESULTS
    # *************************************************************************
    r2_rez = gscv.cv_results_['mean_test_r2']
    mae_rez = -gscv.cv_results_['mean_test_neg_mean_absolute_error']
    rmse_rez = -gscv.cv_results_['mean_test_neg_root_mean_squared_error']
    best_C = gscv.best_params_['SVR__C']
    best_r2 = gscv.best_score_

    # output results
    output["gs"] = gscv
    output["r2"] = r2_rez
    output["mae"] = mae_rez
    output["rmse"] = rmse_rez

    print(f"Best parameter: C = {best_C}, (r² = {best_r2 :.3f})")
    # plot it if asked
    if plot:
        fig = go.Figure()
        x_ = Cs
        fig.add_trace(go.Scatter(x=x_, y=r2_rez, name="r²"))
        fig.add_trace(go.Scatter(x=x_, y=mae_rez, name="MAE"))
        fig.add_trace(go.Scatter(x=x_, y=rmse_rez, name="RMSE"))
        fig.update_xaxes(autorange="reversed")
        fig.update_layout(width=600, height=400,
            title=f"SVR best param: C={best_C} (r²={best_r2 :.3f})")
        fig.show()

    return output


## <a id='toc2_4_'></a>[Chargement des données](#toc0_)

In [4]:
DATASETS_PATH = "./"
dataset_name = "oc_p3_2016_Building_Energy_Benchmarking.csv"
data_raw = pd.read_csv(DATASETS_PATH+dataset_name)

## <a id='toc2_5_'></a>[Nettoyage des données](#toc0_)

Nettoyage des données selon le processus vu dans le notebook précédent (0.01% d'usage non résidentiel minimum) :

In [5]:
non_res_min_usage = 0.0001

df = data_cleaner(data_raw, non_res_min_usage, verbose=False)

Vérification du nettoyage :

In [6]:
describe_df(df)

,tags,count,unique,top,freq,mean,std,min,25%,50%,75%,max,type,nan,nan_pct
AreaPerBldg(sf),,2098.0,2012,-,-,0.000023,0.000017,0.000001,0.00001,0.000021,0.000034,0.000233,float64,0,0.00
AreaPerFloor(sf),,2098.0,2051,-,-,0.000066,0.000046,0.000001,0.000032,0.000054,0.000092,0.000321,float64,0,0.00
BuildingRatio,,2098.0,408,-,-,0.947185,0.119715,0.409128,1.0,1.0,1.0,1.0,float64,0,0.00
ENERGYSTARScore,,1461.0,100,-,-,67.399042,28.237912,1.0,51.0,75.0,91.0,100.0,float64,637,30.36
Electricity_I(kBtu/sf),,2098.0,2097,-,-,37.057358,33.82885,-2.219558,18.21398,26.665662,43.265648,293.115813,float64,0,0.00
GHGEmissionsIntensity,,2098.0,460,-,-,1.307264,1.820165,-0.02,0.27,0.68,1.5,16.99,float64,0,0.00
NaturalGas_I(kBtu/sf),,2098.0,1412,-,-,18.121622,31.391338,0.0,0.0,7.658574,21.235542,306.377567,float64,0,0.00
Neighborhood,📦,2098.0,13,DOWNTOWN,401,-,-,-,-,-,-,-,object,0,0.00
NonResidentialRatio,,2098.0,892,-,-,0.678306,0.383137,0.012124,0.25936,1.0,1.0,1.0,float64,0,0.00
ParkingRatio,,2098.0,979,-,-,0.119609,0.155758,0.0,0.0,0.0,0.232171,0.795862,float64,0,0.00


# <a id='toc3_'></a>[Pré-traitement](#toc0_)

Le pipeline de pré-traitement sert à formatter les divers traitements effectués sur nos données afin de les rendre répétables : les jeux de données d'entraînement et de test passeront donc **exactement par les mêmes étapes**.  

Le pipeline est ainsi un **moyen idéal de comparer plusieurs modèles**.

## <a id='toc3_1_'></a>[Séparation du jeu de données](#toc0_)

Pour garantir un bon **apprentissage sans biais ni fuite de données**, il est important de **séparer le jeu de données en plusieurs échantillons** :
- un échantillon d'entraînement ("train"), qui sert à entraîner un modèle
- un échantillon de validation ("validation"), dont les résultats servent à améliorer l'hyperparamétrage d'un modèle
- un échantillon de test ("test"), resté intouché afin de comparer plusieurs modèles entre eux

Pour tous ces échantillons, on prendra soin d'**écarter la variable cible, ici `GHGEmissionsIntensity`**.

In [7]:
target = "GHGEmissionsIntensity"

X_train, X_val, X_test, y_train, y_val, y_test = X_y_splitter(
    df, target, valid=True, verbose=True)

INFO:root:X.shape = (2098, 13), y.shape = (2098,)
INFO:root:X_train.shape = (1468, 13), y_train.shape = (1468,)
X_val.shape = (315, 13), y_val.shape = (315,)
X_test.shape = (315, 13), y_test.shape = (315,)


## <a id='toc3_2_'></a>[Encodage, normalisation et imputation](#toc0_)

Après avoir défini quelles variables étaient catégorielles et lesquelles étaient numériques (en excluant la cible), il est possible d'appliquer les différentes étapes du pré-traitement :
- **encodage** des variables catégorielles
- **normalisation** des variables numériques
- **imputation** des valeurs nulles

Choix effectués :
- encodage catégoriel one hot : même s'il **augmente la dimension du jeu de données**, cela reste **dans une mesure raisonnable** et **ne risque pas autant de sur-apprentissage que le target encoding**, vu la distribution non gaussienne des données et leur variance parfois élevée ;
- normalisation min-max : on souhaite avoir des **valeurs entre 0 et 1** afin de pouvoir les comparer sur une même échelle, ce qui améliore considérablement bon nombre de modèles de machine learning et l'imputation à venir (kNN, basée sur la distance) ;
- imputation par kNN : le choix s'est porté sur ce type d'imputation pour sa **fiabilité**, d'autant qu'il n'y a qu'**une seule variable aux valeurs manquantes** et toutes les autres peuvent servir d'appui fiable à l'imputation. Elle est basée ici sur **5 voisins, un juste compromis biais / variance**.

In [8]:
classif_cols = ["Neighborhood"]

num_cols = X_train.drop(classif_cols, axis=1).columns.to_list()

In [9]:
# encoding and min-max normalization
col_transf = ColumnTransformer([
        ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
        ('min_max_scaler', MinMaxScaler(), num_cols),
        ],
    remainder = 'passthrough',
    verbose_feature_names_out=False # keep columns names as is
    )

# imputation
imputer = Pipeline([
        ('knn_imputer', KNNImputer(n_neighbors=5)),
    ])

Application sur un échantillon :

In [10]:
# all-in-one preprocessing pipeline example
preprocessor = Pipeline([
        ("col_transf", col_transf),
        ('knn_imputer', KNNImputer(n_neighbors=5)),
    ])

preprocess_test = preprocessor.fit_transform(X_train)

In [11]:
display(preprocess_test)

array([[0.        , 0.        , 0.        , ..., 0.08811606, 0.        ,
        0.17391304],
       [0.        , 0.        , 0.        , ..., 0.04629827, 0.        ,
        0.52173913],
       [0.        , 0.        , 0.        , ..., 0.06016642, 0.        ,
        0.08695652],
       ...,
       [0.        , 0.        , 0.        , ..., 0.18561979, 0.        ,
        0.91304348],
       [0.        , 0.        , 0.        , ..., 0.03712396, 0.        ,
        0.92173913],
       [0.        , 0.        , 0.        , ..., 0.15190953, 0.        ,
        0.87826087]])

Problème : le `ColumnTransformer` transforme notre dataframe en tableau sans nom de colonnes ni index.

Il est donc nécessaire de le reconvertir afin qu'il reste sous la forme d'un dataframe :

In [12]:
pp_test_df = pd.DataFrame(
    preprocess_test,
    columns=preprocessor[0:].get_feature_names_out()
    )
describe_df(pp_test_df)

,tags,count,mean,std,min,25%,50%,75%,max,unique,type,nan,nan_pct
AreaPerBldg(sf),,1468.0,0.097409,0.073631,0.0,0.041502,0.086918,0.146501,1.0,1420,float64,0,0.0
AreaPerFloor(sf),,1468.0,0.205569,0.146364,0.0,0.095964,0.166655,0.286379,1.0,1445,float64,0,0.0
BuildingRatio,,1468.0,0.914602,0.196588,0.0,1.000000,1.000000,1.000000,1.0,284,float64,0,0.0
ENERGYSTARScore,,1468.0,0.651639,0.259315,0.0,0.494949,0.701010,0.858586,1.0,353,float64,0,0.0
Electricity_I(kBtu/sf),,1468.0,0.139674,0.124662,0.0,0.068520,0.101648,0.163223,1.0,1467,float64,0,0.0
NaturalGas_I(kBtu/sf),,1468.0,0.059175,0.103043,0.0,0.000000,0.024594,0.068179,1.0,974,float64,0,0.0
Neighborhood_BALLARD,,1468.0,0.040191,0.196473,0.0,0.000000,0.000000,0.000000,1.0,2,float64,0,0.0
Neighborhood_CENTRAL,,1468.0,0.028610,0.166766,0.0,0.000000,0.000000,0.000000,1.0,2,float64,0,0.0
Neighborhood_DELRIDGE,,1468.0,0.027929,0.164826,0.0,0.000000,0.000000,0.000000,1.0,2,float64,0,0.0
Neighborhood_DOWNTOWN,,1468.0,0.196185,0.397246,0.0,0.000000,0.000000,0.000000,1.0,2,float64,0,0.0


# <a id='toc4_'></a>[Pipeline complet](#toc0_)

Compilation de ce qui a été décidé en amont en différentes fonctions et pipelines.

## <a id='toc4_1_'></a>[Paramétrage et préparation des données](#toc0_)

In [13]:
non_res_min_usage = 0.0001
target = "GHGEmissionsIntensity"
random_state = 0

# data cleaning
df = data_cleaner(data_raw, non_res_min_usage, verbose=False)

# defining columns types
classif_cols = ["Neighborhood"]
num_cols = df.drop(classif_cols, axis=1).columns.to_list()
num_cols.remove(target)

## <a id='toc4_2_'></a>[Définitions](#toc0_)

In [14]:
# encoding and min-max normalization
col_transf = ColumnTransformer([
        ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
        ('min_max_scaler', MinMaxScaler(), num_cols),
        ],
    remainder = 'passthrough',
    verbose_feature_names_out=False
    )

# imputation
imputer = Pipeline([
        ('knn_imputer', KNNImputer(n_neighbors=5)),
    ])

# all-in-one (except cleaning)
preprocessor = Pipeline([
        ("col_transf", col_transf),
        ('knn_imputer', KNNImputer(n_neighbors=5)),
    ])

## <a id='toc4_3_'></a>[Application du preprocessing](#toc0_)

In [15]:
# data split
X_train, X_val, X_test, y_train, y_val, y_test = X_y_splitter(
    df, target, random_state, valid=True)

In [16]:
# PIPELINE -> to dataframe
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
    columns=preprocessor[0:].get_feature_names_out())
X_val = pd.DataFrame(preprocessor.fit_transform(X_val),
    columns=preprocessor[0:].get_feature_names_out())
X_test = pd.DataFrame(preprocessor.fit_transform(X_test),
    columns=preprocessor[0:].get_feature_names_out())

# <a id='toc5_'></a>[Estimateurs](#toc0_)

Nous sommes dans le cas d'une régression avec étiquetage, soit un apprentissage supervisé.  
De nombreux modèles peuvent nous aider dans la résolution de notre problème, voici une sélection de **4 modèles d'apprentissage supervisé simples et pertinents**.

## <a id='toc5_1_'></a>[Mesures comparatives](#toc0_)

Afin de pouvoir comparer chaque modèle, nous allons utiliser plusieurs mesures statistiques et pratiques :
- le **coefficient de détermination linéaire r²** (coefficient de Pearson), qui compare le **rapport entre la somme des carrés de la régression (SSR) et la somme des carrés totale (SST)**, que l'on peut comparer avec la somme des carrés des erreurs (SSE) :  
$$r² = \frac{SSR}{SST} = \frac{\sum_{i=1}^{n} (\hat{y}_i - \overline{y})^2}{\sum_{i=1}^{n} (y_i - \overline{y})^2} = \frac{SST-SSE}{SST} = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \overline{y})^2}$$  

<img src="https://365datascience.com/resources/blog/2018-11-image8-5-1024x495.jpg" title="SST, SSR, SSE" width=400px>  

Ce score a toutefois besoin d'être complété car il fait un comparatif global par rapport aux données, sans pour autant pouvoir expliquer la contribution de telle ou telle variable au résultat.

- la **MAE** (Mean Absolute Error), qui est la **moyenne des écarts absolus entre prévisions et données** :  
$$MAE = \frac{1}{n}\sum_{i=1}^{n} |y_i - \hat{y}_i|$$  
Le fait que chaque erreur ait la même importance fait qu'elle est **moins sensible aux valeurs atypiques** et son optimisation avec la médiane en fait un **indicateur robuste mais parfois biaisé** ("simpliste").

- la **RMSE** (Root Mean Squarred Error) est la **racine carrée de la moyenne des écarts carrés entre prévisions et données** :  
$$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$  
Cette fois, en élevant au carré la valeur des écarts, cet indicateur est **sensible aux valeurs atypiques** car chaque anomalie a un poids important dans le calcul. Cela ainsi que son optimisation par la moyenne en fait un **indicateur fiable mais parfois trop sensible**.

- la **durée de l'entraînement** en secondes, ce qui peut avoir un fort impact avec un jeu de données important et orienter le choix vers tel ou tel modèle.

Création d'un tableau comparatif des différents estimateurs :

In [17]:
scores = pd.DataFrame(index=["r2", "MAE", "RMSE", "Train_time (sec)"])

def rec_score(estimator, sample, predictions, time):
    scores[estimator] = [
        round(r2_score(sample, predictions), 3),
        round(mean_absolute_error(sample, predictions), 3),
        round(mean_squared_error(sample, predictions, squared = False), 3),
        round(time, 3)
        ]
    return scores

## <a id='toc5_2_'></a>[Étalon : dummy regressor](#toc0_)

Nécessaire pour comparer les "vrais" estimateurs, celui-ci est factice et prédit toujours une même valeur, ici la moyenne :

In [18]:
dummy = DummyRegressor(strategy="mean")

# fit
start = time.time()
dummy.fit(X_train, y_train)
end = time.time()
fit_time = end - start

rec_score("Dummy_val", y_val, dummy.predict(X_val), fit_time)
rec_score("Dummy_test", y_test, dummy.predict(X_test), fit_time)

,Dummy_val,Dummy_test
r2,-0.000,-0.003
MAE,1.148,1.038
RMSE,1.818,1.460
Train_time (sec),0.001,0.001


## <a id='toc5_3_'></a>[Linear Regression](#toc0_)

Cet estimateur utilise la **méthode des moindres carrés ordinaire** (*ordinaire* = *non pondéré*), qui consiste en un **ajustement itératif des coefficients de chaque variable** pour **minimiser la somme des carrés des erreues** (SSE).  
À la fin de l'entraînement, il **conserve les coefficients optimaux pour chaque variable** (ou *poids*, que nous retrouverons plus loin dans un tableau comparatif).

[Documentation ici](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [19]:
linreg = LinearRegression()

# fit
start = time.time()
linreg.fit(X_train, y_train)
end = time.time()
fit_time = end - start

rec_score("LinReg_val", y_val, linreg.predict(X_val), fit_time)
rec_score("LinReg_test", y_test, linreg.predict(X_test), fit_time)

,Dummy_val,Dummy_test,LinReg_val,LinReg_test
r2,-0.000,-0.003,0.951,0.413
MAE,1.148,1.038,0.215,0.656
RMSE,1.818,1.460,0.404,1.116
Train_time (sec),0.001,0.001,0.007,0.007


## <a id='toc5_4_'></a>[Support Vector Machines (SVM)](#toc0_)

Le modèle des vecteurs de support (Support Vector Machine) peut être utilisé pour de la **classification (SVC) ou de la régression (SVR), pas nécessairement linéaire**, constituant un ensemble de méthodes d'apprentissage supervisé.  
On les utilise aussi pour la **détection de valeurs aberrantes**.

Pour la classification, il **recherche un hyperplan (frontière de décision) capable de séparer les données en deux classes et maximise la marge** (distance entre cet hyperplan et les observations les plus proches).  
Pour la régression, le principe est similaire mais avec une **fonction de perte (loss) adaptée à la régression** en cherchant à minimiser une erreur de prédiction (hyperparamètre $\epsilon$).  

Comme souvent les données **ne peuvent pas totalement être séparées linéairement, il existe un hyperparamètre pour relativiser l'importance de l'erreur et de la marge** (hyperparamètre $C$).  
Une SVR va donc chercher à **minimiser l'erreur de prédiction tout en maintenant un équilibre entre la maximisation de la marge et la réduction des erreurs**.  

Aussi, dans les régressions non linéaires, un hyperplan peut utiliser des dimensions supérieures :

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/10/SVM_8.png" title="séparation impossible sans hyperplan" width=400px>  

            👇            👇              👇  
          
séparation impossible par un hyperplan avec un noyau linéaire → création d'un hyperplan avec un noyau polynomial : $z = x² + y²$  

            👇            👇              👇  

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/10/SVM_9.png" title="séparation impossible sans hyperplan" width=400px>

Une machine à vecteur de support **ne dépend que d'un sous-ensemble des données d'apprentissage**.  
Là où en classification, la fonction de perte ne se préoccupe pas des points d'apprentissage qui se trouvent au-delà de la marge, dans une régression, la fonction de perte **ignore les échantillons dont la prédiction est proche de leur cible**.

Le modèle `sklearn.svm.SVR` a été utilisé ici.  
Il **effectue en interne une validation croisée à 5 blocs**, coûteuse si les données sont nombreuses.

Documentation  [ici](https://scikit-learn.org/stable/modules/svm.html#regression) et [ici](https://blent.ai/blog/a/svm-support-vector-machine)

> On remarque une augmentation du r² d'environ 5% après application du noyau (*kernel*) linéaire.  

> Bien qu'ils soient très similaires, après tests, le **modèle `sklearn.svm.LinearSVR` aurait dû être utilisé à sa place** car il est **mieux adapté aux problèmes de régression linéaire** en permettant par exemple davantage de flexibilité dans le choix des fonctions de perte.


In [20]:
svr = svm.SVR(kernel='linear')

# fit
start = time.time()
svr.fit(X_train, y_train)
end = time.time()
fit_time = end - start

rec_score("SVM_val", y_val, svr.predict(X_val), fit_time)
rec_score("SVM_test", y_test, svr.predict(X_test), fit_time)

,Dummy_val,Dummy_test,LinReg_val,LinReg_test,SVM_val,SVM_test
r2,-0.000,-0.003,0.951,0.413,0.947,0.513
MAE,1.148,1.038,0.215,0.656,0.273,0.625
RMSE,1.818,1.460,0.404,1.116,0.420,1.017
Train_time (sec),0.001,0.001,0.007,0.007,0.038,0.038


## <a id='toc5_5_'></a>[Random Forest Regressor](#toc0_)

Cet estimateur utilise un **ensemble d'arbres décisionnels individuels, ayant une corrélation très faible**, ce qui permet d'appliquer concrètement le concept de sagesse des foules et protège l'ensemble contre les erreurs individuelles.  

Il utilise le **bagging** (bootstrap aggregation, "mise en sac") : chaque arbre est testé sur un échantillon aléatoire du jeu de données mais de même taille que lui (remplacement).
> Exemple : jeu original = [1, 2, 3, 4, 5, 6] → bagging = [1, 2, 2, 3, 6, 6] (taille 6, avec aléa et remplacement)

Ces "comités de décision" indépendants utilisent un **système de "vote" final** pour prendre une décision.  

[Documentation ici](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [21]:
run_forest = RandomForestRegressor()

# fit
start = time.time()
run_forest.fit(X_train, y_train)
end = time.time()
fit_time = end - start

rec_score("RdmForestReg_val", y_val, run_forest.predict(X_val), fit_time)
rec_score("RdmForestReg_test", y_test, run_forest.predict(X_test), fit_time)

,Dummy_val,Dummy_test,LinReg_val,LinReg_test,SVM_val,SVM_test,RdmForestReg_val,RdmForestReg_test
r2,-0.000,-0.003,0.951,0.413,0.947,0.513,0.955,0.406
MAE,1.148,1.038,0.215,0.656,0.273,0.625,0.215,0.650
RMSE,1.818,1.460,0.404,1.116,0.420,1.017,0.385,1.123
Train_time (sec),0.001,0.001,0.007,0.007,0.038,0.038,3.128,3.128


## <a id='toc5_6_'></a>[Gradient Boosting Regressor](#toc0_)

Le boosting est lui aussi un **modèle d'ensemble additif**.  

Il s'agit d'**entraîner de manière séquentielle une série de modèles de base**, en corrigeant à chaque séquence les **erreurs des modèles précédents grâce à une pondération accrue des observations mal estimées**.  
Cela crée une **grande dépendance entre les modèles** et la performance du modèle final dépend donc de la performance de tous les modèles de base précédents.  
En contrepartie, il y a une **forte réduction du biais**.  

Le **modèle de base du Gradient Boosting est un arbre de régression** utilisant une fonction loss d'erreur quadratique pour ajuster son entraînement.

[Doc ici](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)

In [22]:
boost = GradientBoostingRegressor()

# fit
start = time.time()
boost.fit(X_train, y_train)
end = time.time()
fit_time = end - start

rec_score("GradBoostReg_val", y_val, boost.predict(X_val), fit_time)
rec_score("GradBoostReg_test", y_test, boost.predict(X_test), fit_time)

,Dummy_val,Dummy_test,LinReg_val,LinReg_test,SVM_val,SVM_test,RdmForestReg_val,RdmForestReg_test,GradBoostReg_val,GradBoostReg_test
r2,-0.000,-0.003,0.951,0.413,0.947,0.513,0.955,0.406,0.954,0.465
MAE,1.148,1.038,0.215,0.656,0.273,0.625,0.215,0.650,0.238,0.614
RMSE,1.818,1.460,0.404,1.116,0.420,1.017,0.385,1.123,0.391,1.066
Train_time (sec),0.001,0.001,0.007,0.007,0.038,0.038,3.128,3.128,0.908,0.908


## <a id='toc5_7_'></a>[Scores d'entraînement](#toc0_)

In [23]:
r2_mean_val = round((scores['LinReg_val'][0] + scores['SVM_val'][0]
    + scores['RdmForestReg_val'][0] + scores['GradBoostReg_val'][0]) / 4, 3)
mae_mean_val = round((scores['LinReg_val'][1] + scores['SVM_val'][1]
    + scores['RdmForestReg_val'][1] + scores['GradBoostReg_val'][1]) / 4, 3)
rmse_mean_val = round((scores['LinReg_val'][2] + scores['SVM_val'][2]
    + scores['RdmForestReg_val'][2] + scores['GradBoostReg_val'][2]) / 4, 3)

r2_mean_test = round((scores['LinReg_test'][0] + scores['SVM_test'][0]
    + scores['RdmForestReg_test'][0] + scores['GradBoostReg_test'][0]) / 4, 3)
mae_mean_test = round((scores['LinReg_test'][1] + scores['SVM_test'][1]
    + scores['RdmForestReg_test'][1] + scores['GradBoostReg_test'][1]) / 4, 3)
rmse_mean_test = round((scores['LinReg_test'][2] + scores['SVM_test'][2]
    + scores['RdmForestReg_test'][2] + scores['GradBoostReg_test'][2]) / 4, 3)

scores['Valid_mean'] = [r2_mean_val, mae_mean_val, rmse_mean_val, "-"]
scores['Test_mean'] = [r2_mean_test, mae_mean_test, rmse_mean_test, "-"]

display(scores)

# show r² on a plot
# prepare data
estims = ["LinReg", "SVM", "RdmForest", "GradBoost"]
scor_plt = scores.drop(["Dummy_val", "Dummy_test", "Valid_mean", "Test_mean"],
    axis=1).loc["r2"].T
scor_val = scor_plt.filter(regex="_val$", axis=0)
scor_test = scor_plt.filter(regex="_test$", axis=0)

# create plot
fig = go.Figure(data=[
    go.Bar(name='Validation split', x=estims, y=scor_val),
    go.Bar(name='Test split', x=estims, y=scor_test)
    ])
# add valid mean
fig.add_hline(y=scores["Valid_mean"]["r2"],
    line_dash="dash", line_color="blue")
# add test mean
fig.add_hline(y=scores["Test_mean"]["r2"], line_dash="dash", line_color="red")
# group bars
fig.update_layout(barmode='group', width=700, height=400)

fig.show()

,Dummy_val,Dummy_test,LinReg_val,LinReg_test,SVM_val,SVM_test,RdmForestReg_val,RdmForestReg_test,GradBoostReg_val,GradBoostReg_test,Valid_mean,Test_mean
r2,-0.000,-0.003,0.951,0.413,0.947,0.513,0.955,0.406,0.954,0.465,0.952,0.449
MAE,1.148,1.038,0.215,0.656,0.273,0.625,0.215,0.650,0.238,0.614,0.235,0.636
RMSE,1.818,1.460,0.404,1.116,0.420,1.017,0.385,1.123,0.391,1.066,0.4,1.08
Train_time (sec),0.001,0.001,0.007,0.007,0.038,0.038,3.128,3.128,0.908,0.908,-,-


## <a id='toc5_8_'></a>[Comparatif structurel des échantillons cibles](#toc0_)

In [24]:
compare_splits(y_test, y_val, "y_test", "y_val", plot=True)

,pop,min,max,mean,med,std
y_test,315.0,0.01,10.41,1.238444,0.72,1.459805
y_val,315.0,0.01,15.42,1.332317,0.70,1.821098


On constate des **scores r² très élevés sur l'échantillon de validation** mais des scores plutôt attendus sur l'échantillon de test.  
Comparons les échantillons 

Plusieurs conclusions se dégagent a priori :
- les valeurs obtenues sur l'échantillon de validation suggèrent une **fuite de données** puisqu'elles sont bien trop élevées sans même avoir paramétré les estimateurs
- elle peuvent également être le résultat d'une **trop forte corrélation** entre une variable et la cible
- les écarts de valeurs entre les échantillons test et validation laissent penser à un **sur-apprentissage ou à une très forte sensibilité des estimateurs aux valeurs atypiques**

> Pour plus de praticité dans la suite du notebook, tous les traitements ont été regroupés dans deux fonctions :
> - `process_all()` regroupant tous les traitements du pipeline et qui appelle la fonction
> - `data_cleaner()` pour le nettoyage des données.

# <a id='toc6_'></a>[Explication des scores : recherche](#toc0_)

## <a id='toc6_1_'></a>[Constats initiaux](#toc0_)

On constate en premier lieu un écart très important entre les échantillons de test et de validation après 2 tests simples (effectués en amont) :  
**suppression du filtre de la cible `GHGEmissionsIntensity` <= 20**,  
soit un filtre de **seulement 2 individus de l'échantillon total**, qui sont [comme vu dans l'EDA](./P3_EDA.ipynb) des valeurs atypiques.

1. processus avec **rand_state=0**  
→ le score "y_test" est très élevé alors que le "y_val" est très négatif  
→ on constate une distribution des échantillons cibles très différente, par exemple un **maximum de 7.41 pour y_val et 25.71 pour y_test**

2. processus avec **rand_state=1**  
→ le score "y_val" est très élevé alors que le "y_test" est médiocre (sauf pour l'estimateur SVM)  
→ on constate une distribution des échantillons cibles plus proche, notamment dans les valeurs élevées avec un **maximum de 25.71 pour y_val et 16.38 pour y_test**  

### <a id='toc6_1_1_'></a>[Influence de l'échantillonnage](#toc0_)

On voit que le **random state est très influent** puisqu'il change la répartition des outliers et la distribution des cibles.  
Notamment, un meilleur équilibrage train / val apparaît lorsque les distributions des cibles sont proches, avec notamment l'influence du maximum (valeurs atypiques).

Les estimateurs utilisés sont apparemment fort sensibles à la distribution et nous avons besoin d'effectuer les tests sur plusieurs échantillons afin de faire une évaluation moyenne.

### <a id='toc6_1_2_'></a>[Méthode pas à pas sur 10 random states](#toc0_)

Pour y remédier, nous allons réaliser chaque pas du traitement sur 10 random states différents et sauvegarder les résultats dans une variable contenant :
- le nom du "pas" effectué
- le détail des actions menées
- le découpage et statistiques des cibles train / test pour visualiser les différences entre random states
- le tableau des scores avec métriques r², MAE, RMSE, temps d'entraînement, médiane et moyenne pour chacun

👉 De **très nombreux tests ont ainsi été effectués** (notamment dans un [notebook séparé, visible ici](./P3_data_leak_tests.ipynb)) afin de rechercher d'où peut provenir cette éventuelle fuite de données avec cette méthode pas à pas.

### <a id='toc6_1_3_'></a>[Fuite possible dans le pré-traitement ou dans le pipeline ?](#toc0_)

Rapidement, les recherches **ont écarté tout risque du côté du pipeline** avec des scores sensiblement similaires.  
De plus, elles ont fait apparaître l'**origine dans le processus de nettoyage des données** (pré-traitement), comme nous le verrons en détail ci-dessous.

Il a donc fallu **reprendre pas à pas tout le pré-traitement** défini dans la précédente analyse exploratoire, avec méthode ci-avant pour éviter le biais de l'échantillonnage.  
C'est donc ce qui sera détaillé ci-après.

## <a id='toc6_2_'></a>[Recherche pas à pas : scores (r²) selon les traitements](#toc0_)

Les principaux traitements sont détaillés dans le texte qui suit le graphique ci-dessous qui, lui, représente la moyenne des r² sur 10 échantillons différents.  
Puisque plusieurs échantillonnages sont réalisés, le **découpage des tests suivants est simplifié avec un seul découpage train / test** (plus d'échantillon "validation").

Les détails complets se trouvent [dans un notebook dédié](./P3_data_leak_tests.ipynb).

Ces manipulations consistent une **base de traitement sur laquelle s'incrémentent diverses actions**.  
La base de traitement initiale est composée comme suit :
- **suppression des doublons** (aucun dans le jeu initial),
- suppression des **données non-conformes uniquement** (outliers métier, 32 individus),
- **nettoyage de la variable `Neighborhood`** (formattage et normalisation des noms pour éviter les doublons),
- **suppression des variables inutiles** au pipeline,
- **suppression des valeurs vides** sauf pour la variable `ENERGYSTARScore` (imputée dans le pipeline qui suit),
- **tri des colonnes par ordre alphabétique** pour plus de lisibilité lors des analyses.

👉 Aucune fuite dans cette base de travail, qui permet d'assainir globalement le jeu de données.  

Comme dit plus haut, **tous ces traitements de nettoyage sont documentés et effectués** au sein de la fonction `data_cleaner()` (base + actions postérieures).

Sous le graphique qui suit, on peut **lire les actions effectuées à tel ou tel point** ainsi que **le nombre d'individus filtrés le cas échéant, entre parenthèses en fin de ligne**.

Lorsqu'une voie d'exploration mène à une mauvaise direction, elle est abandonnée pour une autre, elle-même approfondie, etc.  

Les 3 derniers résultats servent à comparer les **influences des traitements effectués sur les variables** de ***Steam use***, ***Electricity*** et ***Natural Gas***.

In [25]:
data_leak_scores = pickle.load(open("data_leak_scores.p", "rb"))
display_all_r2(data_leak_scores)

save_actions = BASE: drop duplicates (0), keep compliants only, clean neighborhood, drop useless feats, drop nans except ENERGYSTARScore, sort columns
save_actions_2    = BASE + filter GHGEmissionsIntensity, NumberofBuildings, NumberofFloors, PropertyGFATotal (22)
save_actions_3    = ........ + create BuildingRatio feature
save_actions_4    = ............ + drop PropertyGFABuilding(s) feature
save_actions_5    = ................ + filter BuildingRatio (19)
save_actions_6    = .................|.. + create ParkingRatio feature
save_actions_6_1  = .................|...... + drop PropertyGFAParking feature
save_actions_6_2  = .................|.......... + filter ParkingRatio (8)
save_actions_7    = ................ + create AreaPerFloor(sf) and AreaPerBldg(sf) features
save_actions_7_1  = .................... + drop NumberofFloors and NumberofBuildings features
save_actions_7_2  = ........................ + filter AreaPerFloor(sf) (1)
save_actions_8    = ............................ + cr

## <a id='toc6_3_'></a>[Interprétation des résultats et retour sur le brief](#toc0_)

On voit une montée moyenne du r² de 0.33 à 0.6 à la seule création des variables structurelles `SteamUse_I(kBtu/sf)`, `Electricity_I(kBtu/sf)` et `NaturalGas_I(kBtu/sf)`.  
De nouveau, il remonte de 0.6 à 0.86 lorsqu'on filtre les variables `SteamUse_I(kBtu/sf)` et `Electricity_I(kBtu/sf)` (12 individus filtrés).

Dans le brief de Douglas, le project lead, il est dit :  
> L'objectif est de **<ins>te passer des relevés de consommation annuels futurs (attention à la fuite de données)</ins>**.  
> Nous ferons de toute façon pour tout nouveau bâtiment un **premier relevé de référence la première année**, donc **<ins>rien ne t'interdit d'en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d'énergie utilisées</ins>**.

L'objectif est de se passer des relevés futurs, mais il est possible d'utiliser les relevés effectués la première année pour en déduire des variables structurelles comme la nature et proportion des sources d'énergie utilisées.  
C'est ce qui a été fait avec les variables `SteamUse_I(kBtu/sf)`, `Electricity_I(kBtu/sf)` et `NaturalGas_I(kBtu/sf)`.

## <a id='toc6_4_'></a>[Conclusions de la recherche de fuite](#toc0_)

**Il ne s'agit donc pas d'une fuite de données** à ce niveau puisque :
- les données utilisées sont **disponibles**
- elles ont été transformées pour définir des **données structurelles**
- le jeu de données est **bien séparé en amont** des traitements

## <a id='toc6_5_'></a>[Échantillonnage : sensibilité aux outliers](#toc0_)

Il se peut donc que la forte sensibilité aux valeurs atypiques constatée précédemment mène à un sur-apprentissage.

Cela peut se reproduire par un comparatif entre deux échantillonnages avec une distribution des données différente et l'impact sur les résultats du r².  
Exemple avec les random states 0 et 9 :

In [26]:
scores_df_model = pd.DataFrame(columns=["rdm_st", "metric", "LinReg", "SVR",
    "RdmForestReg", "GradBoostReg", "MEAN", "MEDIAN"])

rdm_state = 0
scores_rs0 = scores_df_model.copy()
rs0 = process_all(data_raw, non_res_min_usage, rdm_state, target, scores_rs0,
    plot=True)

rdm_state = 9
scores_rs9 = scores_df_model.copy()
rs9 = process_all(data_raw, non_res_min_usage, rdm_state, target, scores_rs9,
    plot=True)

On voit clairement une différence dans la distribution des données échantillonnées avec :
- un random state de 9 (rs9) qui ne contient pas de valeur élevée (max = 10,41)
- tandis que le random state de 0 (rs0)= est plus étalé avec une valeur maximale à 15,42

Cela **change la moyenne et l'écart-type**.  
Les scores du rs9 étant bien plus bas que le rs0, on peut en déduire que **plus les données de test sont étalées, plus le modèle est performant**, avec des écarts très importants.

👉 **Il semble bien y avoir un sur-apprentissage**.  
Pour certains estimateurs, un **hyperparamétrage pourrait réduire** cet effet et dans tous les cas une **boucle d'appprentissage sur plusieurs échantillons est nécessaire**, à l'instar d'une validation croisée.

## <a id='toc6_6_'></a>[Corrélation : feature importance et poids des modèles](#toc0_)

Une autre piste serait une **très forte corrélation entre les variables énergétiques structurelles et la cible**.  
L'EDA précédente avait déjà montré une corrélation très forte avec la variable `NaturalGas_I(kBtu/sf)` (0.95), `Electricity_I(kBtu/sf)` (0.5) et `SteamUse_I(kBtu/sf)` (0.23).

La *feature importance* permet d'évaluer l'importance d'une variable indépendante sur notre modèle, donc **quelles variables contribuent le plus à la performance globale du modèle**.  

La **feature importance par permutation** réalise ceci en **mélangeant aléatoirement les valeurs d'une variable** dans l'ensemble de données de test puis mesure la performance du modèle réentraîné avec cette permutation.  
Les variables sont **ensuite classées en fonction de l'importance** calculée : les plus grandes pertes de performance après la permutation sont considérées comme les plus importantes et vice versa.

Une analyse de la feature importance par modèle permet de voir les variables aux influences marquées et confirmer ou non la corrélation vue dans l'EDA :

In [27]:
permut_fi(rs0)

On peut aussi voir les poids de chaque modèle variable par variable :

In [28]:
_ = pd.DataFrame(columns = X_train.columns)
_.loc["LinReg_coef"] = linreg.coef_
_.loc["SVR_coef"] = svr.coef_[0]
_.loc["RdmForest"] = run_forest.feature_importances_
_.loc["GradBoost"] = boost.feature_importances_
display(_.T)

,LinReg_coef,SVR_coef,RdmForest,GradBoost
Neighborhood_BALLARD,-4.433656e+09,0.012367,0.000061,0.000000e+00
Neighborhood_CENTRAL,-4.433656e+09,-0.000057,0.000010,0.000000e+00
Neighborhood_DELRIDGE,-4.433656e+09,0.000000,0.000005,0.000000e+00
Neighborhood_DOWNTOWN,-4.433656e+09,0.008571,0.000153,3.184032e-06
Neighborhood_EAST,-4.433656e+09,0.003964,0.000039,7.847674e-07
Neighborhood_GREATER DUWAMISH,-4.433656e+09,-0.047445,0.000031,4.352451e-06
Neighborhood_LAKE UNION,-4.433656e+09,-0.008352,0.000074,0.000000e+00
Neighborhood_MAGNOLIA / QUEEN ANNE,-4.433656e+09,0.009728,0.000197,3.048143e-08
Neighborhood_NORTH,-4.433656e+09,-0.008199,0.000013,0.000000e+00
Neighborhood_NORTHEAST,-4.433656e+09,-0.021048,0.000020,1.111792e-05


Nous avons une confirmation de la matrice de corrélation de l'EDA avec une variable qui fait presque toute la variance : `NaturalGas_I(kBtu/sf)`, suivie par `SteamUse_I(kBtu/sf)`, `SiteEUIWN(kBtu/sf)` et `Electricity_I(kBtu/sf)`.  
Ce sont **presque exclusivement les 3 variables énergétiques structurelles crées qui sont responsables de la variance**  

👉 la **corrélation est confirmée et explique en partie les résultats** très élevés constatés ci-avant.  
Il n'y a pas de solution à cet effet, hormis se passer volontairement d'une variable très utile, ce qui serait contre-productif.

## <a id='toc6_7_'></a>[Conclusion de l'explication des scores](#toc0_)

> Rappel : recherches portées sur  
> 1️⃣ **fuite de données**  
> 2️⃣ **corrélation** trop importante entre une variable et la cible  
> 3️⃣ **sur-apprentissage** causé par une forte sensibilité aux valeurs atypiques

Résumé des actions de recherche pouvant expliquer ces résultats surprenants :  
- constat de l'**influence de l'échantillonnage** sur les résultats et correction par une **boucle sur plusieurs random states** pour les tests suivants
- élimination de la piste du pipeline et concentration des **recherches sur le nettoyage des données**
- nombreux tests avec **reprise pas à pas** de tout le processus de pré-traitement
  - influence très forte des **variables énergétiques structurelles**
  - **montée du r²** à la création des variables et à la filtration des individus
- relecture du **brief**
- 1️⃣ conclusion : **pas de fuite de données**
- 2️⃣ **sensibilité aux outliers confirmée**, menant à un **sur-apprentissage**
  - solution possible : **hyperparamétrage**
  - pratique nécessaire : **boucle d'échantillonnage** pour l'apprentissage (type *cross validation*)
- 3️⃣ confirmation de la **forte corrélation** avec `NaturalGas_I(kBtu/sf)`
  - pas de solution sans volonté de baisser la qualité du modèle

# <a id='toc7_'></a>[Choix de l'estimateur et hyperparamétrage](#toc0_)

Sur le graphique des scores r², on voit que les algorithmes de régression linéaires **les plus stables sont SVR et RdmForestRegressor** : ils résistent mieux aux différents traitements et bruits selon les batches aléatoires.  
On constate notamment leur **forte résistance aux valeurs aberrantes**.

L'algorithme **SVR est privilégié** car il est **également l'un des plus rapides** (25 fois plus rapide que GradBoostRegressor et presque 80 fois plus rapide que RdmForestRegressor).

Il est possible d'affiner notre estimateur SVR avec des hyperparamètres afin d'**améliorer ses résultats sur les échantillons avec une distribution différente** de l'échantillon d'entraînement, sur lesquels son r² passait de 0.96 à 0.56 (avec une MAE et une RMSE dont les valeurs triplaient).  

SVR a déjà un hyperparamètre enregistré, à savoir son noyau linéaire. Il n'est pas possible de jouer sur la valeur du noyau pour l'améliorer, les autres valeurs étant plutôt réservées à des régressions d'autres types.  

Cependant, il est possible de jouer sur son **hyperparamètre C, qui gère la régularisation du modèle**, à savoir sa capacité à pondérer le bruit dans les observations. Or le jeu de données a une distribution assez étalée, surtout sur certains échantillons aléatoires.  

Son hyperparamètre C est fixé à 1 au départ et il est possible de **le réduire pour augmenter la régulatisation** de l'estimateur.

Premier test avec un random state de 9, où les scores r² étaient bas (0.56) et la MAE et la RMSE plus hautes :

In [29]:
rdm_state = 9
svr_c_9 = plot_svr_c_scores(rdm_state)

A priori, on retrouve un **r² très important avec un C minimum (0.1)**.

Il serait intéressant de voir les résultats pour d'autres random states, avec une distribution moins différente entre jeu d'entraînement et jeu de test.  
Exemples avec un random state de 0 et de 1 :

In [30]:
rdm_state = 0
svr_c_0 = plot_svr_c_scores(rdm_state)

In [31]:
rdm_state = 1
svr_c_1 = plot_svr_c_scores(rdm_state)

On constate que la **meilleure valeur de C est changeante** selon l'échantillonnage :
- minimale (0.1) pour un random state à 9 avec des scores initiaux très bas (0.56 → 0.94)
- maximale (1.0) pour un random state à 1 avec des scores initiaux très hauts (0.996)
- et intermédiaire (0.3) pour un random state à 0 avec des scores initiaux déjà élevés (0.962 → 0.976)

La RMSE et la MAE sont impactées dans le sens inverse du score r² avec des évolutions bien marquées.

Il serait donc intéressant d'avoir la **meilleure valeur de C sur plusieurs échantillonnages différents**.

## <a id='toc7_1_'></a>[Validation croisée](#toc0_)

La technique GridSearchCV est souvent très appropriée pour tester divers hyperparamètres d'un modèle et s'adapte très bien à ces tests.  
Elle utilise également le r² comme métrique de score pour les régressions, ce qui facilitera les comparatifs.

Voici une **optimisation de C avec une validation croisée à 5 blocs** pour les mêmes 10 valeurs de C (de 1.0 à 0.1).

In [32]:
best_svr_cv = get_best_svr_cv(data_raw, non_res_min_usage, target, plot=True)

Best parameter: C = 1.0, (r² = 0.998)


On voit qu'en généralisant les tests sur des échantillons aléatoires croisés, **la configuration initiale de l'estimateur SVR avec C=1 obtient les meilleurs scores**.  
C'est donc cette configuration qui sera utilisée par la suite.

# <a id='toc8_'></a>[Influence de l'ENERGYSTAR Score](#toc0_)

Lors de la feature importance et de l'analyse du poids des modèles ci-avant, il **avait été constaté que l'influence de la variable "ENERGYSTARScore" était presque nulle**.  

Afin de le vérifier à nouveau de manière stable, il est possible de **comparer la même validation croisée que précédemment** avec un argument qui permet de **retirer cette variable du jeu de données** avant les pipelines de machine learning :

In [33]:
best_svr_cv_no_ess = get_best_svr_cv(data_raw, non_res_min_usage, target, ESS=False, plot=True)

Best parameter: C = 1.0, (r² = 0.998)


Il n'y a aucun changement dans les trois métriques : **l'ENERGYSTAR Score n'a pas d'influence sur ce modèle**.

# <a id='toc9_'></a>[Comparatif avec ratio rés / non-rés](#toc0_)

Le ratio d'utilisation résidentielle et non résidentielle créé précédemment permet un **ajustement du modèle par les professionnels du domaine**.

Voici ce qu'on constate en le paramétrant **entre une utilisation non résidentielle minimale à 0.01% et 100%** (échelle logarithmique) :

In [34]:
non_res_usages = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 0.75, 1]
# prepare scores lists
r2 = [] ; mae = [] ; rmse = []

for u in non_res_usages:
    # run process
    r = get_best_svr_cv(data_raw, u, target, plot=False)

    # get max score index
    r2s = best_svr_cv_no_ess["r2"].tolist()
    i = r2s.index(max(r2s))

    # add best scores to scores lists
    r2.append(r["r2"][i])
    mae.append(r["mae"][i])
    rmse.append(r["rmse"][i])

# get best param
best_r2 = max(r2)
best_i = r2.index(best_r2)
best_nru = non_res_usages[best_i]

# plot it
fig = go.Figure()
x_ = non_res_usages
fig.add_trace(go.Scatter(x=x_, y=r2, name="r²"))
fig.add_trace(go.Scatter(x=x_, y=mae, name="MAE"))
fig.add_trace(go.Scatter(x=x_, y=rmse, name="RMSE"))
fig.update_xaxes(type="log")
fig.update_layout(width=600, height=400,
    title=f"SVR best param: non_res={best_nru} (r²={best_r2 :.3f})")
fig.show()

Best parameter: C = 1.0, (r² = 0.998)
Best parameter: C = 1.0, (r² = 0.998)
Best parameter: C = 1.0, (r² = 0.998)
Best parameter: C = 1.0, (r² = 0.998)
Best parameter: C = 1.0, (r² = 0.997)
Best parameter: C = 1.0, (r² = 0.995)
Best parameter: C = 1.0, (r² = 0.995)
Best parameter: C = 1.0, (r² = 0.995)


Que ce soit pour le r², la MAE ou la RMSE, l'évolution peut sembler anecdotique car elle est située entre deux valeurs très proches, mais elle est **tout de même visible** et il est impaginable qu'un impact soit plus important avec un autre modèle ou d'autres données.

# <a id='toc10_'></a>[Conclusion](#toc0_)

La **très forte corrélation entre la variable structurelle définissant une consommation de gaz naturel par pied carré** et la cible (émissions de gaz à effet de serre) fausse le modèle et lui fait obtenir des résultats prédictifs presque parfaits.

Pour être certain de **se prémunir contre une fuite de données qui entraînerait la nécessaire refonte de cette étude**, il est important de **vérifier** avec Douglas le project lead **que nous pouvons bien obtenir un relevé de consommation de gaz la première année** pour créer cette variable structurelle.

Par ailleurs, plusieurs autres points sont à vérifier avec Douglas :
- **usage non résidentiel** : une fonction d'ajustement a été créée mais ce n'est que pour palier le manque d'information à ce sujet, qui nécessite un point avec les équipes métier pour déterminer un seuil et améliorer le remplissage des données en amont
- il **manque des informations cruciales concernant les parkings** : en effet, selon qu'il s'agit par exemple d'un parking sous-terrain, avec ventilation ou non, à l'air libre, éclairé ou non, la consommation de ce dernier change fortement et peut avoir un impact très fort, surtout au vu des importantes surfaces qu'ils occupent
- **SourceEUI est une variable intéressante qui pourrait mener à une étude plus détaillée de la consommation générale de la ville pour réduire son impact**, car elle prend en compte la consommation du site avec tout l'impact de la production et de la distribution de cette énergie : à quel point est-elle fiable ? détaillée ? comment est-elle obtenue ?